<a href="https://colab.research.google.com/github/mc-ivan/computer-vision/blob/main/lab1/notebook/Practica1_Grupo3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 1: Conteo de seres vivos mediante Visión Artificial Clásica con OpenCV

**Grupo 3**
- Ivan Mamani
- Yesica Luna
- Elvis Miranda
- Karem Huacota
- Karen Torrico


## Caso 1: Conteo de Atletas que cruzan la linea de meta

In [ ]:
# Librerias
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# ===========================
# 1. Cargar video desde GitHub
# ===========================
video_url = "https://github.com/mc-ivan/computer-vision/raw/main/lab1/sources/atletas.mp4"
video_path = "/content/atletas.mp4"

!wget -O {video_path} {video_url}

cap = cv2.VideoCapture(video_path)

# Propiedades del video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# ===========================
# 2. Parámetros
# ===========================
line_position = frame_width // 2      # Línea vertical al centro
min_contour_area = 3000               # Filtrado por área
max_match_distance = 30               # Distancia para tracking

# ===========================
# 3. Video de salida
# ===========================
out_path = "/content/atletas_conteo.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(out_path, fourcc, fps, (frame_width, frame_height))

# ===========================
# 4. Inicialización
# ===========================
ret, frame_prev = cap.read()
if not ret:
    raise RuntimeError("No se pudo leer el video")

gray_prev = cv2.cvtColor(frame_prev, cv2.COLOR_BGR2GRAY)
gray_prev = cv2.GaussianBlur(gray_prev, (5,5), 0)
cv2_imshow(gray_prev)

counter = 0
objects = []  # cada objeto: [cx, cy, crossed]

# ===========================
# 5. Procesamiento frame a frame
# ===========================
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)

    # Diferencia de frames
    diff = cv2.absdiff(gray_prev, gray)

    # Umbralización
    _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
    # cv2_imshow(thresh) # comentado porque satura la memoria

    # Eliminación de ruido
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Contornos
    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    centroids_current = []

    # ===========================
    # 6. Filtrado de contornos
    # ===========================
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_contour_area:
            continue

        x, y, w, h = cv2.boundingRect(cnt)

        aspect_ratio = h / float(w)
        if aspect_ratio < 1.0:
            continue

        if y < frame_height * 0.2 or y > frame_height * 0.95:
            continue

        cx = x + w // 2
        cy = y + h // 2
        centroids_current.append((cx, cy))

        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.circle(frame, (cx, cy), 5, (0,0,255), -1)

    # ===========================
    # 7. Tracking + conteo
    # ===========================
    for cx, cy in centroids_current:
        matched = False

        for obj in objects:
            px, py, crossed = obj

            if abs(cx - px) < max_match_distance and abs(cy - py) < max_match_distance:
                matched = True
                obj[0], obj[1] = cx, cy

                # Cruce de derecha a izquierda (UNA sola vez)
                if not crossed and px > line_position >= cx:
                    counter += 1
                    obj[2] = True
                break

        if not matched:
            objects.append([cx, cy, False])

    # Limpiar objetos fuera de escena
    objects = [obj for obj in objects if 0 < obj[0] < frame_width]

    # ===========================
    # 8. Visualización
    # ===========================
    cv2.line(
        frame,
        (line_position, 0),
        (line_position, frame_height),
        (255,0,0),
        2
    )

    cv2.putText(
        frame,
        f"Cruzo la linea de meta: {counter}",
        (20, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0,0,255),
        2
    )

    out.write(frame)

    if frame_number % 30 == 0:
        cv2_imshow(frame)

    gray_prev = gray.copy()

# ===========================
# 9. Finalización
# ===========================
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video procesado correctamente:")
print(out_path)


In [ ]:
# Re-codificar a formato compatible con Colab y mostrar video final
!ffmpeg -y -i /content/atletas_conteo.mp4 -vcodec libx264 -pix_fmt yuv420p /content/atletas_conteo_ok.mp4

from IPython.display import Video

Video("/content/atletas_conteo_ok.mp4", embed=True)


# Caso 2: Vacas cruzando el rio

In [ ]:
# Conteo de vacas cruzando el río (de izquierda a derecha)
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Video
video_url = "https://github.com/mc-ivan/computer-vision/raw/main/lab1/sources/vacas.mp4"
video_path = "/content/vacas.mp4"
!wget -O {video_path} {video_url}

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Línea vertical a un tercio desde el lado derecho
line_position = frame_width - (frame_width // 3)
min_contour_area = 7000
max_match_distance = 60

# Video de salida
out_path = "/content/vacas_conteo.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(out_path, fourcc, fps, (frame_width, frame_height))

# Inicialización
ret, frame = cap.read()
if not ret:
    raise RuntimeError("No se pudo leer el video")

counter = 0
objects = []  # [cx, cy, crossed]

# -------------------------
# Subtractor de fondo MOG2
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1

    # Sustracción de fondo
    fgmask = fgbg.apply(frame)
    _, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    fgmask = cv2.dilate(fgmask, None, iterations=2)

    # Contornos
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    centroids_current = []

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_contour_area:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = h / float(w)
        if aspect_ratio > 1.0:  # Rectángulos horizontales
            continue

        cx = x + w // 2
        cy = y + h // 2
        centroids_current.append((cx, cy))

        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.circle(frame, (cx, cy), 5, (0,0,255), -1)

    # Tracking y conteo
    for cx, cy in centroids_current:
        matched = False
        for obj in objects:
            px, py, crossed = obj
            if abs(cx - px) < max_match_distance and abs(cy - py) < max_match_distance:
                matched = True
                obj[0], obj[1] = cx, cy
                if not crossed and px < line_position <= cx:
                    counter += 1
                    obj[2] = True
                break
        if not matched:
            objects.append([cx, cy, False])

    objects = [obj for obj in objects if 0 < obj[0] < frame_width]

    # Visualización
    cv2.line(frame, (line_position,0), (line_position,frame_height), (0,0,255), 3)
    cv2.putText(frame, f"Cruce del rio: {counter}", (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    out.write(frame)
    if frame_number % 30 == 0:
        cv2_imshow(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print("Video procesado correctamente:", out_path)


In [ ]:
# Re-codificar a formato compatible con Colab y mostrar video final
!ffmpeg -y -i /content/vacas_conteo.mp4 -vcodec libx264 -pix_fmt yuv420p /content/vacas_conteo_ok.mp4

from IPython.display import Video

Video("/content/vacas_conteo_ok.mp4", embed=True)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab